# TORCH.AUTOGRAD 에 대한 간단한 소개

torch.autograd 는 신경망 학습을 지원하는 PyTorch의 자동 미분 엔진이다. 이 단원에서는 autograd가 신경망 학습을 어떻게 돕는지에 대한 개념적 이해를 할 수 있다.

# Background

신경망은 어떤 입력 데이터에 대해 실행되는 중첩된 함수들의 모음이다. 이 함수들은 PyTorch에서 Tensor로 저장되는 weight와 bias로 구성된 매개변수들로 정의된다.  

신경망을 학습하는 것은 2단계로 구성된다.

1. Forward propagation : 순전파 단계에서, 신경망은 정답을 맞추기 위해 최선의 추측을 한다. 이렇게 추측을 하기 위해 입력데이터를 각 함수들에서 실행한다.

2. Backward Propagation : 역전파 단계에서, 신경망은 추측한 값에서 발생한 loss에 비례하여 매개변수들을 적절히 조절한다. 출력으로부터 역방향으로 이동하며 오류에 대한 함수들의 매개변수들의 gradient(미분값, 변화도)를 수집하고, gradient descent를 사용하여 매개변수들을 최적화한다. 

# PyTorch에서 사용법

학습 단계를 보자.  
torchvision에서 미리 학습된 resnet18 모델을 불러온다. 3채널짜리 높이와 넓이가 64인 이미지 하나를 표현하는 무작위의 데이터 텐서를 생성 후, 이에 상응하는 label(정답)을 무작위 값으로 초기화한다. 미리 학습된 모델의 정답은 (1,1000)의 shape을 갖는다.

In [1]:
import torch, torchvision
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

/Users/junghwankim/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/junghwankim/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/junghwankim/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Input data를 모델의 각 layer에 통과시켜 prediction을 생성

In [3]:
prediction = model(data) # forward-pass

모델의 prediction value와 그에 해당하는 label을 사용하여 error(loss)를 계산한다. 다음 단계는 신경망을 통해 이 에러를 역전파 하는 것이다. error tensor에 .backward()를 호출하면 역전파가 시작된다. 그 다음 Autograd가 매개변수의 grad 속성에 모델의 각 매개변수에 대한 gradient를 계산하고 저장한다.

In [4]:
loss = (prediction - labels).sum()
loss.backward() # backward-pass

다음으로 optimizer를 불러온다. 이 예제에서는 Learning rate 0.1과 momentum 0.9를 갖는 SGD(stochastic gradient descent)를 사용한다. 

optimizer에 모델의 모든 매개변수를 등록한다.

In [5]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

마지막으로 .step()을 호출하여 gradient descent를 시작한다.

optimizer는 .grad에 저장된 gradient에 따라 각 매개변수를 조정한다.

In [7]:
optim.step() # gradient descent

# Autograd differentiation

autograd가 어떻게 gradient를 수집하는지 살펴보겠다.

requires_grad = True를 갖는 2개의 Tensor a 와 b를 만든다.

requires_grad = True는 autograd에 모든 operation들을 추적해야 한다고 알려준다.

In [8]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

이제 a,b로부터 새로운 텐서 Q를 만든다.

In [9]:
Q = 3*a**3 - b**2

이제 a,b가 NN의 매개변수이고 Q가 error라고 가정하겠다.

신경망을 학습할 때, 아래와 같이 매개변수들에 대한 error의 gradient를 구해야 한다.  
![](1.png)  
Q에 대해서 .backward()를 호출할 때, autograd는 이러한 변화도들을 계산하고 이를 각 텐서의 .grad attribute에 저장한다.

Q는 vector이므로 Q.backward()에 gradient argument를 명시적으로 전달해야 한다. gradient는 Q와 같은 shape의 텐서로 Q 자기 자신에 대한 gradient를 나타낸다.
즉, 아래의 그림과 같다.
![](2.png)

동일하게, Q.sum().backward()와 같이 Q를 스칼라 값으로 집계(aggregate)한 뒤 암시적으로 .backward()를 호출할 수 도 있다.


In [10]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

이제 gradient는 a.grad, b.grad에 저장된다.

In [11]:
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])
